### CP13 - SQLite 

**Task 1**

Write Python code to create the tables `Book` and `Author` with the following fields in the database `library.db`.

```
Table: BOOK
. ID INTEGER AUTO INCREMENT (Primary Key)
. NAME TEXT
. AUTHORID INTEGER  

Note: AUTHORID is a Foreign Key that is link to the Primary key of Author table
```

```
Table: AUTHOR
. ID INTEGER AUTO INCREMENT (Primary Key)
. NAME TEXT
```


In [57]:
import sqlite3

con = sqlite3.connect("assets/library.db")
con.row_factory = sqlite3.Row

cur = con.cursor()
con.execute("""CREATE TABLE BOOK(
ID INTEGER PRIMARY KEY AUTOINCREMENT,
NAME TEXT,
AUTHORID INTEGER
)""")

con.execute("""CREATE TABLE AUTHOR(
ID INTEGER PRIMARY KEY AUTOINCREMENT,
NAME TEXT
)""")

cur.close()
con.close()

**Task 2**

Write a program that read the file `books.txt` and store the name of book in the `Book` table and name of the author in the `Author` table.

If there are no author information, replace the author as "unknown".

In [8]:
import sqlite3

def addBook(con, cur, name, author):
    # Get AuthorID OR create new Author object
   
    cur.execute("SELECT ID FROM AUTHOR WHERE NAME=?", (author,))
    result = cur.fetchall()
    
    authorID = 0
    if len(result) > 0:
        authorID = result[0]['ID']
    else:
        cur.execute("""
            INSERT INTO AUTHOR(NAME) VALUES(?)
        """, (author,) )
        con.commit()
    
    # Insert Book
    
    cur.execute("""
        INSERT INTO BOOK(NAME, AUTHORID) VALUES(?, ?)
    """, (name, authorID))
    con.commit()
    

con = sqlite3.connect("assets/library.db")
con.row_factory = sqlite3.Row

cur = con.cursor()

f = open("assets/books.txt", 'r')
books = [("TEST", "TEST2")]
for line in f:
    segments = line.split(' by ')
    name = segments[0].strip('"').strip()
    author = segments[1].strip() if len(segments) > 1 else "Unknown"
    addBook(con, cur, name, author)
    
#con.close()
#cur.close()

**Task 3**
    
Write a program to print out a menu that allow user to choose the function they would like to perform.

Menu:

Please select a function to perform:

[a] Add a book

[b] Update Book

[c] Update Author

[d] Delete Book

[e] Print Book List BY Author

[q] Quit


1. Add a book - User will input the book's name and author's name of the book to be added. If the book or author exists, they should not be added in the database, otherwise new records for both book and author will be inserted.


2. Update Book- User will input part or full book's name. 
   With the part or full book's, the program will list out the books (Book ID, Book Name) and prompt the user to select the
   a book ID to update. The program will prompt and allow the book's name to be updated.
   
   
3. Update Author - User will input part or full author's name.
   With the part or full author name, the program will list out the authors (Author ID, Author Name) and prompt the user to select the author ID to update. The program will prompt and allow the author's name to be updated.


4. Delete a book by Name  -  User will input part or full book's name,  the program will list out and prompt the user to select the book ID to be deleted. There is no need to delete the author's record.


5. Print books by Author - User will input part or full author's name, the program will list out the authors and prompt the user to select the author ID. The program will list the books that matches the author.


6. Quit - Exit from the program



In [ ]:
menu_string = """
========================
1. Add Book
2. Update Book
3. Update Author
4. Delete Book
5. Print Books by Author
6. Quit
========================
""".strip()

def getDB():
    con = sqlite3.connect("assets/library.db")
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    return (con, cur)

def main():
    running = True
    while running:
        print(menu_string)
        option = int(input("Enter Option: "))
        
        match option:
            case 1:
                # Input name and author, then add to database
                name = str(input("Enter Name: "))
                author = str(input("Enter Author Name: "))
                con, cur = getDB()
                addBook(con, cur, name, author)
                
            case 2:
                # Input partial book name, confirm bookID, enter new book name to push to database
                name = str(input("Enter Partial/Full Book Name: "))
                con, cur = getDB()
                cur.execute("SELECT ID, NAME FROM BOOK WHERE NAME LIKE ?", (f"%{name}%", ))
                result = cur.fetchall()
                
                for row in result:
                    print(f"[{row["ID"]}: {row["NAME"]}]")
                
                bookID = int(input("Enter Book ID: "))
                newName = input("Enter new Book Name: ")
                cur.execute("""
                    UPDATE BOOK
                    SET NAME = ?
                    WHERE ID = ?
                """, (newName, bookID))
                con.commit()
                
            case 3:
                # Input partial author name, confirm authorID, enter new author name to push to database
                name = str(input("Enter Partial/Full Author Name: "))
                con, cur = getDB()
                cur.execute("SELECT ID, NAME FROM AUTHOR WHERE NAME LIKE ?", (f"%{name}%", ))
                result = cur.fetchall()
                
                for row in result:
                    print(f"[{row["ID"]}: {row["NAME"]}]")
                
                authorID = int(input("Enter Author ID: "))
                newName = input("Enter new Author Name: ")
                cur.execute("""
                    UPDATE AUTHOR
                    SET NAME = ?
                    WHERE ID = ?
                """, (newName, authorID))
                con.commit()
                
            case 4:
                # Input partial book name, confirm bookID, then delete Book
                name = str(input("Enter Partial/Full Book Name: "))
                con, cur = getDB()
                cur.execute("SELECT ID, NAME FROM BOOK WHERE NAME LIKE ?", (f"%{name}%", ))
                result = cur.fetchall()
                
                for row in result:
                    print(f"[{row["ID"]}: {row["NAME"]}]")
                
                bookID = int(input("Enter Book ID: "))
                cur.execute("""
                    DELETE FROM AUTHOR
                    WHERE ID = ?
                """, (bookID))
                con.commit()
                
            case 5:
                # Input partial author name. confirm authorID, print all book names from Author
                name = str(input("Enter Partial/Full Author Name: "))
                con, cur = getDB()
                cur.execute("SELECT ID, NAME FROM AUTHOR WHERE NAME LIKE ?", (f"%{name}%", ))
                result = cur.fetchall()
                
                for row in result:
                    print(f"[{row["ID"]}: {row["NAME"]}]")
                
                authorID = int(input("Enter Author ID: "))
                cur.execute("""
                    SELECT NAME
                    FROM BOOK
                    WHERE AUTHORID = ?
                """, (authorID))
                result = cur.fetchall()
                
                for row in result:
                    print(f"> {row["NAME"]}")
                
            case 6:
                running = False
            case _:
                print("Unknown Option: {option}, please try again. ")
                
        cur.close()
        con.close()

main()